In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

In [0]:
path_to_data = '/content/drive/My Drive/compling_project_data'

In [0]:
import numpy as np

In [0]:
from sklearn.externals import joblib

In [0]:
ohe = joblib.load(os.path.join(path_to_data, 'OneHotEncoder.pkl'))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.19.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
le = joblib.load(os.path.join(path_to_data, 'LabelEncoder.pkl'))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


X - 10-граммы символов (LabelEncoded)

In [0]:
X = np.load(os.path.join(path_to_data, "X.pkl"))

y - следующие символы (OneHot-Encoded)

In [0]:
y = np.load(os.path.join(path_to_data, "y.pkl"))

готовимся обмазываться нейросетями:

In [0]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Dropout

Using TensorFlow backend.


Загружаем мэппинг энкодера скопипастев из консоли на локалхосте, раунд!

In [0]:
classes_mapping = ['', '\t', '\n', '\r', '\x1d', ' ', '!', '"', '#', '$', '%', '&',
       "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3',
       '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
       'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
       'u', 'v', 'w', 'x', 'y', 'z', '~', '\x81', '\x85', '\x8a', '\x93',
       '\x94', '\x99', '¡', '§', '©', 'ª', '«', '®', '°', '±', '´', 'µ',
       '¼', '½', '¾', 'É', '×', 'ç', 'è', 'é', 'ï', 'û', 'ν', 'ο', 'τ',
       'υ', 'ω', '–', '—', '‘', '’', '“', '”', '•', '…', 'Ω', '−',
       '\uf6d9', '�']

In [0]:
dict_len = classes_mapping.size

In [0]:
dict_len

140

Обычный RNN:

In [0]:
X.shape, y.shape

((1841751, 10), (1841751, 1, 140))

In [0]:
y = np.array([i[0] for i in y])

In [0]:
model = Sequential()
model.add(Embedding(dict_len, 10, input_length=10))
model.add(SimpleRNN(100))
model.add(Dropout(0.1))
model.add(Dense(dict_len, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
1841751/1841751 [==============================] - 243s 132us/step - loss: 2.2154
Epoch 2/20
1841751/1841751 [==============================] - 241s 131us/step - loss: 2.0324
Epoch 3/20
1841751/1841751 [==============================] - 241s 131us/step - loss: 1.9971
Epoch 4/20
1841751/1841751 [==============================] - 242s 131us/step - loss: 1.9824
Epoch 5/20
1841751/1841751 [==============================] - 240s 130us/step - loss: 1.9753
Epoch 6/20
1841751/1841751 [==============================] - 241s 131us/step - loss: 1.9707
Epoch 7/20
1841751/1841751 [==============================] - 242s 131us/step - loss: 1.9673
Epoch 8/20
1841751/1841751 [==============================] - 245s 133us/step - loss: 1.9668
Epoch 9/20
1841751/1841751 [==============================] - 242s 132us/step - loss: 1.9675
Epoch 10/20
1841751/1841751 [==============================] - 242s 131us/step - loss: 1.9678
Epoch 11/20
1841751/1841751 [==============================] - 245s 1

Посмотрим на качество обученной модели:

In [0]:
model.evaluate(X, y)

1841751/1841751 [==============================] - 87s 47us/step


1.9190844068383655

Категориальная кросс-энтропия равна 1.92 - 

In [0]:
classes_mapping_inv = {classes_mapping[i]:i for i in range(classes_mapping.size)}

In [0]:
active_features = np.array([i for i in range(140)])

In [0]:
def generate_text(text_length):
  start_seq = np.array([[classes_mapping_inv['~'] for i in range(10)]])
  outp = ''
  for i in range(text_length):
    #print(start_seq)
    next_symb = model.predict(start_seq).dot(active_features).astype(int)[0]
    start_seq[0][-1] = next_symb
    outp += classes_mapping[next_symb.astype(int)]
  return outp

In [0]:
generate_text(100)

'Fcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjgkcjg'

In [0]:
from keras.layers import LSTM

In [0]:
model2 = Sequential()
model2.add(Embedding(dict_len, 140, input_length=10))
model2.add(LSTM(100))
model.add(Dropout(0.05))
model.add(Dense(dict_len, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
1841751/1841751 [==============================] - 307s 167us/step - loss: 2.3599
Epoch 2/20
1841751/1841751 [==============================] - 305s 166us/step - loss: 2.1160
Epoch 3/20
1841751/1841751 [==============================] - 315s 171us/step - loss: 2.0627
Epoch 4/20
  53184/1841751 [..............................] - ETA: 5:02 - loss: 2.0444